- 데이터 파일을 읽어 질문과 ID를 추출
- InferSent 모델 및 Google USE 임베딩 함수 준비
- 분류기(MLP) 모델 로드
- 질문을 배치 단위로 임베딩 생성 → 분류기 예측 → 질문 유형(q_type) 리스트 생성
- 결과를 원본 데이터에 반영하여 새 파일로 저장

In [ ]:
# SQuAD 데이터 형식
## - data: 문서 리스트
##   - title: 문서 제목
##   - paragraphs: 문서 내 단락 리스트
##     - context: 단락 텍스트
##     - qas: 단락 내 질문 리스트
##       - question: 질문 텍스트
##       - id: 질문 ID
##       - answers: 정답 리스트
##         - text: 정답 텍스트
##         - answer_start: 정답 시작 위치(인덱스)

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from __future__ import absolute_import, division
from tensorflow.python.framework.ops import enable_eager_execution
enable_eager_execution()
import io
import os
import sys
import logging
import json
import torch
import nltk
import numpy as np
import re
from infersent import InferSent
import tensorflow.compat.v1 as tf
tf.logging.set_verbosity(0)
import tensorflow_hub as hub
# nltk.download('punkt_tab')

# Set PATHs
PATH_SENTEVAL = '../SentEval'  # specify SentEval root if not installed
PATH_TO_DATA = ''  # not necessary for inference
MODEL_VERSION = 1
# 300차원의 사전 단어 임베딩 벡터
PATH_TO_W2V = '../SentEval/glove/glove.840B.300d.txt' if MODEL_VERSION == 1 else '../SentEval/fasttext/crawl-300d-2M.vec'
# 사전 학습된 임베딩 모델 Infersent1
MODEL_PATH = "../SentEval/encoder/infersent%s.pkl" % MODEL_VERSION
V = 1 # version of InferSent

## glove.txt: 각 단어를 300차원의 벡터로 표현하는 사전 임베딩 파일
# ->infersent모델이 입력 문장 임베딩할 때, 각 단어 벡터로 변환에 사용
## infersent1.pkl: 사전 학습된 문장 임베딩 모델의 가중치 파일
# ->여러 단어 임베딩(GloVe)받아 문장 전체의 의미를 하나의 벡터로 변환
# -> GloVe:단어를 벡터로 / InferSent:문장을 벡터로

sys.path.insert(0, PATH_SENTEVAL)
import senteval
from senteval.tools.classifier import MLP

# 경로(폴더 위치), 현재 작업 경로
# os.getcwd()
# 모듈 찾을 때 참조하는 경로 리스트, 상대 경로 해석(sys.path.insert로 수정)
# sys.path

2025-09-15 14:25:16.056037: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def loadFile(fpath):
    """
    MRQA 스타일 데이터 파일을 읽어 질문과 질문 ID를 추출
    파일 한 줄씩 읽어 JSON으로 파싱->각 질문 토큰화하여 저장
    """
    qa_data = []
    qa_ids = []
    # tgt2idx = {'ABBR': 0, 'DESC': 1, 'ENTY': 2,
    #             'HUM': 3, 'LOC': 4, 'NUM': 5}
    with io.open(fpath, 'r', encoding='utf-8') as f:
        for example in f:
            if "header" in json.loads(example):
                continue
            paragraph = json.loads(example)
            for qa in paragraph['qas']:
                qa_data.append(qa['question'].split())
                qa_ids.append(qa['qid'])
    return qa_data, qa_ids

def loadSQuAD(fpath):
    """
    SQuAD 스타일 데이터 파일을 읽어 질문과 질문 ID를 추출
    파일 한 줄씩 읽어 JSON으로 파싱->각 질문 토큰화하여 저장
    """
    qa_data = []
    qa_ids = []
    # tgt2idx = {'ABBR': 0, 'DESC': 1, 'ENTY': 2,
    #             'HUM': 3, 'LOC': 4, 'NUM': 5}
    with io.open(fpath, 'r', encoding='utf-8') as f:
        input_data = json.load(f)["data"]
    for entry in input_data:
        for paragraph in entry["paragraphs"]:
            for qa in paragraph["qas"]:
                qa_data.append(qa['question'].split())
                qa_ids.append(qa['id'])
    return qa_data, qa_ids

In [3]:
# MRQA style input
# file_path = '../data/mrqa/train/HotpotQA.jsonl'
# 질문과 질문 ID 불러오기
# all_qs, all_q_ids = loadFile(file_path)
# SQuAD style input
file_path = '../data/squad/train-v1.1.json'
# 질문과 질문 ID 불러오기
all_qs, all_q_ids = loadSQuAD(file_path)
all_qs, all_q_ids = all_qs[:500], all_q_ids[:500]

print('squad question example: ', all_qs[0])
print('squad question id example: ', all_q_ids[0])

q_type = []
print('File:', file_path)
print('len: ', len(all_q_ids))

squad question example:  ['To', 'whom', 'did', 'the', 'Virgin', 'Mary', 'allegedly', 'appear', 'in', '1858', 'in', 'Lourdes', 'France?']
squad question id example:  5733be284776f41900661182
File: ../data/squad/train-v1.1.json
len:  500


In [ ]:
def clean_sentence(text: str) -> str:
    # 소유격, 축약형 's 분리
    text = re.sub(r"(\w+)'s", r"\1 's", text)
    # 축약형 단순 변환
    text = text.replace("n't", " not")
    text = text.replace("'re", " are")
    text = text.replace("'ve", " have")
    text = text.replace("'ll", " will")
    text = text.replace("'d", " would")
    text = text.replace("'m", " am")
    text = text.replace("What's", "What is") 
    # 하이픈 단어 분리
    text = re.sub(r"(\w+)-(\w+)", r"\1 \2", text)
    # 불필요한 특수문자 제거
    text = re.sub(r"[^A-Za-z0-9\s\?]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text
def preprocess_sentences(sent_list):
    return [clean_sentence(s) for s in sent_list]

def prepare(params, samples):
    # InferSent 모델에 사용할 단어 사전을 구축하는 함수
    # samples: 문장(질문) 리스트
    # 각 문장을 띄어쓰기로 합쳐서 build_vocab에 전달
    
    # 축약형·소유격·하이픈 결합 단어 전처리 
    sentences = [' '.join(sent) if sent != [] else '.' for sent in samples]
    sentences = preprocess_sentences(sentences)
    params['infersent'].build_vocab(sentences, tokenize=False) 

def batcher(params, batch):
    # 입력된 질문 배치를 임베딩 벡터로 변환하는 함수
    # 1. 각 질문(토큰 리스트)을 문자열로 합침
    sentences = [' '.join(sent) if sent != [] else '.' for sent in batch]
    # 2. 축약형·소유격·하이픈 결합 단어 전처리
    sentences = preprocess_sentences(sentences)
    # 3. InferSent 모델로 임베딩 생성
    embeddings1 = params['infersent'].encode(sentences, bsize=params['classifier']['batch_size'], tokenize=False)
    # 4. Google USE 인코더용으로 문장 준비 (빈 문장은 '.'으로 대체)
    batch = [' '.join(sent) if sent != [] else '.' for sent in batch]
    # 5. 축약형·소유격·하이픈 결합 단어 전처리
    batch = preprocess_sentences(batch)
    # 6. Google Universal Sentence Encoder로 임베딩 생성
    embeddings2 = params['google_use'](batch)
    
    # 7. 두 임베딩을 합쳐서 반환 (문장 의미를 더 풍부하게 표현)
    # return np.concatenate((embeddings1, embeddings2), axis=-1)
    return embeddings1

def getEmbeddings(qa_data, params):
    # 전체 질문 리스트를 배치 단위로 나눠 임베딩을 생성하는 함수
    out_embeds = []
    # 배치 크기만큼 반복하며 임베딩 생성
    for ii in range(0, len(qa_data), params['classifier']['batch_size']):
        batch = qa_data[ii:ii + params['classifier']['batch_size']]
        # batcher 함수로 임베딩 생성
        embeddings = batcher(params, batch)
        out_embeds.append(embeddings)
    # 모든 배치 임베딩을 하나로 합쳐 반환(행렬 세로 방향으로 이어붙임)
    # 입력: [batch_size개의 문장] -> 출력: (batch_size, emb_dim)
    return np.vstack(out_embeds) # (N, 4096+512)

In [ ]:
params_senteval = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 5}
params_senteval['classifier'] = {'nhid': 512, 'optim': 'rmsprop', 'batch_size': 16,
                                 'tenacity': 5, 'epoch_size': 4}
params_model = {'bsize': 16, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))
model.set_w2v_path(PATH_TO_W2V)
params_senteval['infersent'] = model.cuda().eval()

def make_embed_fn(module):
    embed = hub.load(module)
    f = embed.signatures["default"]
    return lambda x: f(tf.constant(x))["default"]

encoder = make_embed_fn("https://tfhub.dev/google/universal-sentence-encoder-large/3")
params_senteval['google_use'] = encoder

- 전처리 추가

In [12]:
qa_data = []
qa_ids = []
with io.open(file_path, 'r', encoding='utf-8') as f:
    input_data = json.load(f)["data"]
    input_data = input_data[42:43]
for entry in input_data:
    for paragraph in entry["paragraphs"]:
        for qa in paragraph["qas"]:
            qa_data.append(qa['question'].split())
            qa_ids.append(qa['id'])

In [13]:
batch = all_qs[0:10]
sentences = [' '.join(sent) if sent != [] else '.' for sent in batch]
sentences_clean = preprocess_sentences(sentences)
sentences_clean[:3]

['To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'What is in front of the Notre Dame Main Building?',
 'The Basilica of the Sacred heart at Notre Dame is beside to which structure?']

- Embedding1

In [14]:
params_senteval['infersent'].build_vocab(sentences_clean, tokenize=False)
embeddings1 = params_senteval['infersent'].encode(sentences_clean, bsize=params_senteval['classifier']['batch_size'], tokenize=False, verbose=True)

Found 54(/63) words with w2v vectors
Vocab size : 54
Nb words kept : 98/128 (76.6%)
Speed : 27.1 sentences/s (gpu mode, bsize=16)


In [20]:
qs = all_qs[0:10]
prepare(params_senteval, qs)
embeddings1 = getEmbeddings(qs, params_senteval)
embeddings1.shape

Found 54(/63) words with w2v vectors
Vocab size : 54


(10, 4096)

- Embedding2

In [19]:
embeddings2 = params_senteval['google_use'](sentences)
embeddings2 = embeddings2.numpy()
embeddings2.shape

(10, 512)

In [23]:
out_embeds = []
embeddings = np.concatenate((embeddings1, embeddings2), axis=-1)
out_embeds.append(embeddings)
embeddings.shape

(10, 4608)

In [24]:
# TREC Classifier MLP 로드
clf = MLP(params_senteval['classifier'], inputdim=4096+512, nclasses=6, batch_size=16)
# 학습된 가중치 불러오기
clf.model.load_state_dict(torch.load('../model/qc4qa_model.pth'))  
# 모델을 GPU에 올리고 평가 모드로 변경
clf.model.eval()
out = clf.predict(embeddings)

/tmp/ipykernel_2223245/287952112.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  clf.model.load_state_dict(torch.load('../model/qc4qa_model.pth'))


In [25]:
q_type = []
q_type += np.array(out).squeeze().astype(int).tolist()
q_type

[3, 2, 2, 2, 2, 5, 5, 2, 5, 5]

### Function ###

In [3]:
def clean_sentence(text: str) -> str:
    # 소유격, 축약형 's 분리
    text = re.sub(r"(\w+)'s", r"\1 's", text)
    # 축약형 단순 변환
    text = text.replace("n't", " not")
    text = text.replace("'re", " are")
    text = text.replace("'ve", " have")
    text = text.replace("'ll", " will")
    text = text.replace("'d", " would")
    text = text.replace("'m", " am")
    text = text.replace("What's", "What is") 
    # 하이픈 단어 분리
    text = re.sub(r"(\w+)-(\w+)", r"\1 \2", text)
    # 불필요한 특수문자 제거
    text = re.sub(r"[^A-Za-z0-9\s\?]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def preprocess_sentences(sent_list):
    return [clean_sentence(s) for s in sent_list]

def loadFile(fpath):
    """
    MRQA 스타일 데이터 파일을 읽어 질문과 질문 ID를 추출
    파일 한 줄씩 읽어 JSON으로 파싱->각 질문 토큰화하여 저장
    """
    qa_data = []
    qa_ids = []
    # tgt2idx = {'ABBR': 0, 'DESC': 1, 'ENTY': 2,
    #             'HUM': 3, 'LOC': 4, 'NUM': 5}
    with io.open(fpath, 'r', encoding='utf-8') as f:
        for example in f:
            if "header" in json.loads(example):
                continue
            paragraph = json.loads(example)
            for qa in paragraph['qas']:
                qa_data.append(qa['question'].split())
                qa_ids.append(qa['qid'])
    return qa_data, qa_ids

def loadSQuAD(fpath):
    """
    SQuAD 스타일 데이터 파일을 읽어 질문과 질문 ID를 추출
    파일 한 줄씩 읽어 JSON으로 파싱->각 질문 토큰화하여 저장
    """
    qa_data = []
    qa_ids = []
    # tgt2idx = {'ABBR': 0, 'DESC': 1, 'ENTY': 2,
    #             'HUM': 3, 'LOC': 4, 'NUM': 5}
    with io.open(fpath, 'r', encoding='utf-8') as f:
        input_data = json.load(f)["data"]
        input_data = input_data ################ 개수 줄이기 ################ [:30]
    for entry in input_data:
        for paragraph in entry["paragraphs"]:
            for qa in paragraph["qas"]:
                qa_data.append(qa['question'].split())
                qa_ids.append(qa['id'])
    return qa_data, qa_ids

def prepare(params, samples):
    # InferSent 모델에 사용할 단어 사전을 구축하는 함수
    # samples: 문장(질문) 리스트
    
    # 축약형·소유격·하이픈 결합 단어 전처리 
    sentences = [' '.join(sent) if sent != [] else '.' for sent in samples]
    sentences = preprocess_sentences(sentences)
    params['infersent'].build_vocab(sentences, tokenize=False) # 모델 불러옴

def batcher(params, batch):
    # 입력된 질문 배치를 임베딩 벡터로 변환하는 함수
    # 1. 각 질문(토큰 리스트)을 문자열로 합침
    sentences = [' '.join(sent) if sent != [] else '.' for sent in batch]
    # 2. 축약형·소유격·하이픈 결합 단어 전처리
    sentences = preprocess_sentences(sentences)
    # 3. InferSent 모델로 임베딩 생성
    embeddings1 = params['infersent'].encode(sentences, bsize=params['classifier']['batch_size'], tokenize=False)
    # 4. Google Universal Sentence Encoder로 임베딩 생성
    embeddings2 = params['google_use'](sentences)
    embeddings2 = embeddings2.numpy()
    # 5. 두 임베딩을 합쳐서 반환 (문장 의미를 더 풍부하게 표현)
    return np.concatenate((embeddings1, embeddings2), axis=-1)

def make_embed_fn(module):
    embed = hub.load(module)
    f = embed.signatures["default"]
    return lambda x: f(tf.constant(x))["default"]

def getEmbeddings(qa_data, params):
    # 전체 질문 리스트를 배치 단위로 나눠 임베딩을 생성하는 함수
    out_embeds = []
    # 배치 크기만큼 반복하며 임베딩 생성
    for ii in range(0, len(qa_data), params['classifier']['batch_size']):
        batch = qa_data[ii:ii + params['classifier']['batch_size']]
        # batcher 함수로 임베딩 생성
        embeddings = batcher(params, batch)
        out_embeds.append(embeddings)
    # 모든 배치 임베딩을 하나로 합쳐 반환(행렬 세로 방향으로 이어붙임)
    # 입력: [batch_size개의 문장] -> 출력: (batch_size, emb_dim)
    return np.vstack(out_embeds) # (N, 4096+512)

def updateFile(fpath, q_type, q_ids):
    # MRQA 스타일 데이터 파일에 질문 유형(q_type)을 추가하는 함수
    paragraphs = []
    # 원본 파일을 한 줄씩 읽어서 JSON 객체로 파싱
    with io.open(fpath, 'r', encoding='utf-8') as f:
        for example in f:
            # header가 포함된 줄은 건너뜀
            if "header" in json.loads(example):
                continue
            paragraph = json.loads(example)
            paragraphs.append(paragraph)
    
    total_idx = 0
    # 각 paragraph의 qas 리스트를 순회하며 질문 ID(qid)와 분류 결과(q_type)를 매칭
    for paragraph in paragraphs:
        for qa in paragraph['qas']:
            # 예측된 질문 유형을 해당 질문에 추가
            if qa['qid'] == q_ids[total_idx]:
                qa['q_type'] = q_type[total_idx]
                total_idx += 1
            else:
                # 질문 ID가 맞지 않으면 경고 출력
                print('Can not match qid:', q_ids[total_idx])
    
    # 새로운 파일로 결과 저장 (원본 파일명에서 .jsonl을 _classified.jsonl로 변경)
    with open(fpath[:-6]+'_classified_tmp.jsonl', 'w') as f:
        for sample in paragraphs:
            f.write(json.dumps(sample)+'\n')
    f.close()

def updateSQuAD(fpath, q_type, q_ids):
    # SQuAD 스타일 데이터 파일에 질문 유형(q_type)을 추가하는 함수
    with io.open(fpath, 'r', encoding='utf-8') as f:
        input_data = json.load(f)["data"]
        input_data = input_data ################ 개수 줄이기 ################
    total_idx = 0
    # SQuAD 구조에 맞게 각 질문에 대해 분류 결과를 추가
    for entry in input_data:
        for paragraph in entry["paragraphs"]:
            for qa in paragraph['qas']:
                # 예측된 질문 유형을 해당 질문에 추가
                if qa['id'] == q_ids[total_idx]:
                    qa['q_type'] = q_type[total_idx]
                    total_idx += 1
                else:
                    # 질문 ID가 맞지 않으면 경고 출력
                    print('Can not match qid:', q_ids[total_idx])
    
    # 새로운 파일로 결과 저장 (원본 파일명에서 .json을 _classified.json로 변경)
    with open(fpath[:-5]+'_classified_tmp.json', 'w') as f:
        f.write(json.dumps({"data": input_data})+'\n')
    f.close()


In [4]:
# Set
encoder = make_embed_fn("https://tfhub.dev/google/universal-sentence-encoder-large/3")
params_senteval = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 5}
params_senteval['classifier'] = {'nhid': 512, 'optim': 'rmsprop', 'batch_size': 16,
                                 'tenacity': 5, 'epoch_size': 4}
params_senteval['google_use'] = encoder
params_model = {'bsize': 16, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))
model.set_w2v_path(PATH_TO_W2V)
params_senteval['infersent'] = model.cuda().eval()

# Set Parameter
file_path = '../data/squad/train-v1.1.json'
all_qs, all_q_ids = loadSQuAD(file_path)
q_type = []
clf = MLP(params_senteval['classifier'], inputdim=4096+512, nclasses=6, batch_size=16)
clf.model.load_state_dict(torch.load('../model/qc4qa_model.pth'))  
clf.model.eval()

I0000 00:00:1757913921.869194 2248967 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21876 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:02:00.0, compute capability: 8.9


/tmp/ipykernel_2248967/1828915119.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH))
/tmp/ipykernel_2248967/1828915119.py:19:

Sequential(
  (0): Linear(in_features=4608, out_features=512, bias=True)
  (1): Dropout(p=0.0, inplace=False)
  (2): Sigmoid()
  (3): Linear(in_features=512, out_features=6, bias=True)
)

In [ ]:
with torch.no_grad(): 
    # 질문을 배치 단위로 임베딩->MLP예측->결과 누적
    for i in range(0, len(all_qs), 1000): 
        qs = all_qs[i:1000+i] # 현재 배치에 해당하는 질문 1000개 선택
        prepare(params_senteval, qs) # InferSent모델에 맞게 단어 사전 구축
        # 선택된 질문을 InferSent와 Google USE로 임베딩 벡터로 변환
        embeds = getEmbeddings(qs, params_senteval) 
        # 임베딩된 질문을 MLP 분류기에 입력하여 질문 유형 예측
        out = clf.predict(embeds)
        # 예측 결과를 리스트로 변환하여 전체 결과(q_type)에 추가
        q_type += np.array(out).squeeze().astype(int).tolist()

In [8]:
# SQuAD sytle update
updateSQuAD(file_path, q_type, all_q_ids)

len input_data: 442


In [9]:
file_path = '../data/squad/train-v1.1_classified_tmp.json'
with io.open(file_path, 'r', encoding='utf-8') as f:
    tmp_data = json.load(f)['data']
tmp_data[0]['paragraphs'][0].keys()

dict_keys(['context', 'qas'])

In [14]:
tmp_data[441]['paragraphs'][0]['qas']

[{'answers': [{'answer_start': 105, 'text': 'Nepal'}],
  'question': 'What country is Kathmandu the capital of?',
  'id': '57359bbcdc94161900571ee9',
  'q_type': 3},
 {'answers': [{'answer_start': 332, 'text': 'Sub-Metropolitan City'}],
  'question': 'What does Upa-Mahanagar mean in English?',
  'id': '57359bbcdc94161900571eea',
  'q_type': 3},
 {'answers': [{'answer_start': 615, 'text': 'tri-city'}],
  'question': 'Along with "KTM," what is another nickname of Kathmandu?',
  'id': '57359bbcdc94161900571eeb',
  'q_type': 1},
 {'answers': [{'answer_start': 704, 'text': '975,453'}],
  'question': 'How many people lived in Kathmandu in 2011?',
  'id': '57359bbcdc94161900571eec',
  'q_type': 1},
 {'answers': [{'answer_start': 725, 'text': '49.45'}],
  'question': 'How many square kilometers in size is Kathmandu?',
  'id': '57359bbcdc94161900571eed',
  'q_type': 1}]